<a href="https://colab.research.google.com/github/wksmirnowa/compling_homeworks/blob/master/HW03_Symspell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1.

## Реализация алгоритма SymSpell

In [55]:
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
from string import punctuation
from difflib import get_close_matches
import textdistance

start = '<start>'
end = '<end>'


punctuation += "«»—…“”"
punct = set(punctuation)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i + n]))
    return ngrams


def get_threegrams(sentences):
    threegrams = Counter()

    for sentence in sentences:
        threegrams.update(ngrammer(sentence, n=3))
    return threegrams


def get_threegrams_from_text(text=""):
    sentences = [normalize(t) for t in sent_tokenize(text)][:1000]
    return get_threegrams(sentences)

In [0]:
good = open('/content/drive/My Drive/correct.txt').read()
bad = open('/content/drive/My Drive/mistakes.txt').read()

In [0]:
def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word \
                       in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


In [0]:
for text in open('/content/drive/My Drive/corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    corpus = [normalize(sent) for sent in sents]

In [0]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
N = sum(WORDS.values())
def P(word, N=N):
    return WORDS[word] / N

In [0]:
def n_deletes_helper(word, min_len=0):
    result = set()
    if len(word) > min_len:
        for i in range(len(word)):
            result.add(word[:i] + word[i+1:])
    else:
        result.add(word)
    return result

def n_deletes(word, edit_distance=2, min_len=2):
    edit_distance -= 1
    forms = list(n_deletes_helper(word, min_len))
    tmp = set(forms)
    while edit_distance > 0:
        edit_distance -= 1
        new_forms = []
        for form in tmp:
            new_forms.extend(n_deletes_helper(form, min_len))
        tmp = set(new_forms)
        forms.extend(tmp)
        
    return forms

In [0]:
frequencies_dict = {} 
for sent in corpus:
    for word in sent:
        if word not in frequencies_dict:
            frequencies_dict[word] = P(word)
        else:
            frequencies_dict[word] += 1

In [0]:
frequencies_dict

In [0]:
forms_for_word_dict = {}  
for key in frequencies_dict.keys():
    word_with_del = n_deletes(key)
    for word in word_with_del:
        if word not in forms_for_word_dict.keys():
            forms_for_word_dict[word] = key

In [0]:
forms_for_word_dict

In [66]:
def algoritm_for_words(word: str):
    wordforms_keys = []
    candidates = {}

    if word in frequencies_dict.keys():  # если слово есть в словаре норм слов, возвращаем его
        return word
    else:  # иначе – удаляем по 1 символу
        wordforms = n_deletes(word)  # список из форм с удаленной буквой
        for wordform in wordforms:
            if wordform in forms_for_word_dict.keys():
                wordforms_keys.append(forms_for_word_dict.get(
                    wordform))  # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список

        if wordforms_keys:
            for candidate in wordforms_keys:  # проходимся по потенциальным кандидатам
                if candidate not in candidates:  # если потенциальный кандидат не в списке кандидатов, то
                    candidates[candidate] = P(candidate)  # добавляем его в словарь кандидатов
            candidates_list = list(candidates.items())
            sorted_values = sorted(candidates_list, key=lambda i: i[1])
            if len(sorted_values) > 1:
              if sorted_values[-1][1] == sorted_values[-2][1]:
                edited_word = sorted_values[-2][0]
                return edited_word
              else:
                edited_word = sorted_values[-1][0]
                return edited_word
            else:
                edited_word = sorted_values[-1][0]
                return edited_word

        else:
            print("Слова «" + word + "» нет в нашем словаре, мы не можем его исправить")
            return word


algoritm_for_words('акер')

'актер'

In [0]:
def algoritm_for_text(text: str):

    correct_words = []
    corrected_words = []
    whole_text = []
    with open("corrected.txt", "w+") as f:

      normtext = normalize(text)
      for word in normtext:
        if word in frequencies_dict.keys():  #если слово есть в словаре норм слов, возвращаем его
          whole_text.append(word)
          correct_words.append(word)
          f.write(word + "\n")
        else:  #иначе – удаляем по 1 символу
          wordforms = n_deletes(word) #список из форм с удаленной буквой
          keys_for_wordform = []
          for wordform in wordforms:
            if wordform in forms_for_word_dict.keys():
              keys_for_wordform.append(forms_for_word_dict.get(wordform)) # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список 

          if keys_for_wordform:
              candidates = {}
              for candidate in keys_for_wordform: #проходимся по потенциальным кандидатам
                if candidate not in candidates:
                  candidates[candidate] = P(candidate) #добавляем его в словарь кандидатов
          
              candidates_list = list(candidates.items())
              sorted_values = sorted(candidates_list, key=lambda i: i[1])
              edited_word = sorted_values[-1][0]
              corrected_words.append(edited_word)
              whole_text.append(edited_word)
              f.write(word + "\n")
      
          else:
            whole_text.append(word)
            f.write(word + "\n")
            # print("Слова «" + word + "» нет в нашем словаре, мы не можем его исправить")

    f.close()
    print("Правильные слова", correct_words)
    print("Исправленные слова", corrected_words)
    print("Текст", whole_text)
    return whole_text
    
algoritm_for_text(bad)

Посмотрим, уменьшилась ли доля ошибок по сравнению с изначальным плохим текстом

In [0]:
corrected = open("/content/drive/My Drive/corrected.txt").read()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
mistakes = []
total = 0

word_pairs = align_words(good, bad)

for pair in word_pairs:
  if pair[0] != pair[1]:
    mistakes.append(pair)
  total += 1

In [45]:
print('Доля ошибок - ', len(mistakes)/total)

Доля ошибок -  0.269030969030969


In [0]:
newmistakes = []
newtotal = 0

new_word_pairs = align_words(good, corrected)

for newpair in new_word_pairs:
  if newpair[0] != newpair[1]:
    newmistakes.append(pair)
  total += 1

In [48]:
print('Доля ошибок - ', len(newmistakes)/total )

Доля ошибок -  0.1345154845154845


Доля ошибок уменьшилась, но незначительно, поскольку, во-первых, изначально ошибок в тексте было не очень много, а во-вторых, тот корпус, на основе которого составлялся словарь верных слов для исправлений, был не очень большой и не включал в себя все слова, содержащиеся в текстах good и bad.

Оценим другими метриками:

In [0]:
alignes = [align_words(good, bad)]

In [73]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for align in alignes:
    good_word = [token[0] for token in align]
    bad_word = [token[1] for token in align]
    corrected_text = algoritm_for_text(bad)
    for i in range(len(corrected_text)):
        if  corrected_text[i] == good_word[i]:
            correct += 1
        total += 1
        
        if good_word[i] == bad_word[i]:
            total_correct += 1
            if good_word[i] != corrected_text[i]:
                correct_broken += 1
        else:
            total_mistaken += 1
            if good_word[i] == corrected_text[i]:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

Правильные слова ['для', 'в', 'в', 'что', 'на', 'и', 'в', 'по', 'по', 'в', 'и', 'по', 'на', 'в', 'на', 'на', 'в', 'что', 'на', 'в', 'в', 'в', 'на', 'для', 'будто', 'в', 'и', 'в', 'и', 'и', 'в', 'был', 'на', 'и', 'на', 'на', 'что', 'для', 'и', 'в', 'и', 'на', 'так', 'и', 'что', 'у', 'в', 'на', 'что', 'у', 'на', 'и', 'о', 'был', 'что', 'и', 'на', 'на', 'и', 'и', 'самый', 'и', 'на', 'в', 'и', 'что', 'по', 'так', 'и', 'в', 'и', 'о', 'по', 'и', 'и', 'так', 'и', 'в', 'в', 'на', 'что', 'и', 'в', 'в', 'в', 'на', 'на', 'он', 'на', 'и', 'так', 'и', 'и', 'на', 'в', 'в', 'и', 'и', 'его', 'и', 'и', 'у', 'него', 'был', 'о', 'и', 'и', 'и', 'на', 'второй', 'и', 'он', 'и', 'он', 'в', 'и', 'по', 'и', 'и', 'о', 'фильмов', 'в', 'по', 'и', 'в', 'на', 'по', 'и', 'в', 'до', 'что', 'в', 'был', 'и', 'на', 'и', 'так', 'для', 'на', 'в', 'том', 'что', 'в', 'своим', 'в', 'этом', 'на', 'и', 'и', 'о', 'он', 'в', 'и', 'он', 'в', 'в', 'что', 'что', 'что', 'для', 'в', 'у', 'на', 'в', 'в', 'и', 'на', 'в', 'у', 'его', 'и

# Задание 2

## Реализация этого алгоритма с триграммной моделью

In [82]:
bad = ". ".join(open('/content/drive/My Drive/mistakes.txt').read().splitlines()[:10])
true = ". ".join(open('/content/drive/My Drive/correct.txt').read().splitlines()[:10])

def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word \
                       in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


corpus = []
for text in open('/content/drive/My Drive/corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

N = sum(WORDS.values())


def P(word, N=N):
    return WORDS[word] / N


threegramms = get_threegrams_from_text(true)

vocabulary_words_with_frequency = {}
for sent in corpus:
    for word in sent:
        if word not in vocabulary_words_with_frequency:
            vocabulary_words_with_frequency[word] = P(word)
        else:
            vocabulary_words_with_frequency[word] += 1

vocabulary_wrong_form_with_variants = {}
vocabulary_words_with_threegrams = {}
for key in vocabulary_words_with_frequency.keys():
    for sym in key:
        word_with_del = key.replace(sym, '')
        if word_with_del in vocabulary_wrong_form_with_variants:
            vocabulary_wrong_form_with_variants[word_with_del].append(key)
        else:
            vocabulary_wrong_form_with_variants[word_with_del] = [key]
    for tg in threegramms:
        try:
            if re.search(r"\b" + key + r"\b", tg) is not None:
                if key in vocabulary_words_with_threegrams:
                    vocabulary_words_with_threegrams[key].append(tg)
                else:
                    vocabulary_words_with_threegrams[key] = [tg]
        except Exception:
            # непонятно, как автоматически экранировать спецсимволы
            print('# Ключ ломает все: ' + key)
            break


def algoritm(word):
    wordforms = []
    wordforms_keys = []
    candidates = {}

    if word in vocabulary_words_with_frequency:  # если слово есть в словаре норм слов, возвращаем его
        return [word]
    else:  # иначе – удаляем по 1 символу
        for sym in word:
            word_with_del = word.replace(sym, '')
            wordforms.append(word_with_del)  # добавляем полученные словоформы в список
        for wordform in wordforms:
            if wordform in vocabulary_wrong_form_with_variants:  # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список
                wordforms_keys.extend(vocabulary_wrong_form_with_variants[wordform])

        for candidate in wordforms_keys:  # проходимся по потенциальным кандидатам
            if candidate not in candidates:  # если потенциальный кандидат не в списке кандидатов, то
                candidates[candidate] = P(candidate)  # добавляем его в словарь кандидатов
            else:
                candidates[candidate] += 1

        sorted_values = sorted(candidates, key=candidates.get)
        return sorted_values


def run(text):
    correct_text = []
    words = normalize(text)
    i = 0
    len_words = len(words)
    # print(words)
    while i < len_words - 2:
        local_words = [words[i], words[i + 1], words[i + 2]]
        check_if_bad_word = -1
        for index, word in enumerate(local_words):
            if word not in vocabulary_words_with_frequency:
                check_if_bad_word = index
                break
        if check_if_bad_word != -1:
            # print('BAD: ' + local_words[check_if_bad_word])
            candidates = algoritm(local_words[check_if_bad_word])
            is_fixed = False
            # print('Candidates: ' + str(candidates))
            for cand in candidates:
                local_words[check_if_bad_word] = cand
                tg = " ".join(local_words)
                if cand in vocabulary_words_with_threegrams and tg in vocabulary_words_with_threegrams[cand]:
                    correct_text.extend(local_words)
                    is_fixed = True
                    # print("Успех! Заменили " + words[i + check_if_bad_word] + " на " + local_words[check_if_bad_word])
                    break
            if not is_fixed:
                local_words[check_if_bad_word] = words[i + check_if_bad_word]
                # print('!!! Не починили ' + local_words[check_if_bad_word])
                correct_text.extend(local_words)
            i = i + 3
        else:
            correct_text.append(local_words[0])
            i = i + 1

    return correct_text


run(bad)

# Ключ ломает все: inc.(аффилированная
# Ключ ломает все: главредства(вначале
# Ключ ломает все: lofficiel),но
# Ключ ломает все: у***ном
# Ключ ломает все: u(1
# Ключ ломает все: su(3
# Ключ ломает все: su(2
# Ключ ломает все: конкордия"(верхний
# Ключ ломает все: сегодня(10.11.2018


['симпатичнейшое',
 'шпионское',
 'устройство',
 'такой',
 'себе',
 'гламурный',
 'фотоаппарат',
 'девушки',
 'бонда',
 'миниатюрная',
 'модель',
 'камеры',
 'superheadz',
 'clap',
 'camera',
 'опофеозом',
 'дня',
 'для',
 'меня',
 'сегодня',
 'стала',
 'фраза',
 'услышанная',
 'в',
 'новостях',
 'пояним',
 'эту',
 'мысль',
 'полчатся',
 'вот',
 'такие',
 'язычки',
 'в',
 'массе',
 'своей',
 'они',
 'конечно',
 'все',
 'очень',
 'милые',
 'нащщот',
 'чавеса',
 'разве',
 'что',
 'не',
 'соглашусь',
 'многие',
 'сетуют',
 'на',
 'отсутствие',
 'живого',
 'взаимодействия',
 'между',
 'учеником',
 'и',
 'учителем',
 'а',
 'в',
 'чем',
 'оно',
 'по',
 'сути',
 'основая',
 'цель',
 'мероприятия',
 'практическая',
 'отработка',
 'навыков',
 'по',
 'оказанию',
 'помощи',
 'гражданам',
 'попавшим',
 'в',
 'дтп',
 'а',
 'также',
 'повышение',
 'и',
 'совершенствование',
 'уровня',
 'профессиональной',
 'подготовки',
 'сотрудников',
 'мчс',
 'при',
 'проведении',
 'аварийно-спасательных',
 'работ

Оценим метриками

In [84]:
alignes = [align_words(good, bad)]

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for align in alignes:
    good_word = [token[0] for token in align]
    bad_word = [token[1] for token in align]
    corrected_text = run(bad)
    for i in range(len(corrected_text)):
        if  corrected_text[i] == good_word[i]:
            correct += 1
        total += 1
        
        if good_word[i] == bad_word[i]:
            total_correct += 1
            if good_word[i] != corrected_text[i]:
                correct_broken += 1
        else:
            total_mistaken += 1
            if good_word[i] == corrected_text[i]:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8770491803278688
0.16666666666666666
0.0


Триграммная модель, судя по метрикам, сработала намного лучше.